# Crypto bot metrics evaluation:

## Test data extraction from Binance

In [1]:
from src.get_historical_data import get_historical_data

LOOKBACK = 100

df = get_historical_data(coin='BTCUSDT',lookback=LOOKBACK)
df

,date,BTCUSDT
0,2024-11-27,95863.11
1,2024-11-28,95643.98
2,2024-11-29,97460.00
3,2024-11-30,96407.99
4,2024-12-01,97185.18
...,...,...
95,2025-03-02,94270.00
96,2025-03-03,86220.61
97,2025-03-04,87281.98
98,2025-03-05,90606.01


### Plotting data results:

In [2]:
import plotly.express as px
px.line(df, x='date', y='BTCUSDT')

## Extracting exchange interest from YFinance

In [3]:
from src.get_treasury_rate import get_treasury_rate

df2 = get_treasury_rate(lookback=LOOKBACK)
df2

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


,date,treasury_rate
0,2024-11-26,4.395
1,2024-11-27,4.385
2,2024-11-29,4.373
3,2024-12-02,4.380
4,2024-12-03,4.357
...,...,...
62,2025-02-28,4.193
63,2025-03-03,4.193
64,2025-03-04,4.195
65,2025-03-05,4.205


## Testing metrics

In [4]:
from src.crypto_metrics import CryptoMetrics
metrics = CryptoMetrics(lookback=LOOKBACK)
rsi = metrics.calculate_rsi(df=df, coin='BTCUSDT')

### RSI:

In [5]:
fig = px.line(rsi, x='date', y='BTCUSDT', title='BTCUSDT RSI', labels={'BTCUSDT': 'BTCUSDT Price', 'date': 'Date'})

# Add green dots for rsi_buy_BTCUSDT
fig.add_scatter(x=rsi[rsi['rsi_buy_BTCUSDT'] == 1]['date'], 
                y=rsi[rsi['rsi_buy_BTCUSDT'] == 1]['BTCUSDT'], 
                mode='markers', 
                marker=dict(color='green', size=10), 
                name='RSI Buy')

# Add red dots for rsi_sell_BTCUSDT
fig.add_scatter(x=rsi[rsi['rsi_sell_BTCUSDT'] == 1]['date'], 
                y=rsi[rsi['rsi_sell_BTCUSDT'] == 1]['BTCUSDT'], 
                mode='markers', 
                marker=dict(color='red', size=10), 
                name='RSI Sell')

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/4/46/Bitcoin.svg",
        xref="paper", yref="paper",
        x=1, y=1.2,
        sizex=0.15, sizey=0.15,
        xanchor="right", yanchor="bottom"
    )
)
fig.show()

### Treasury Rate correlation:

In [6]:
from importlib import reload
import src.crypto_metrics
reload(src.crypto_metrics)
from src.crypto_metrics import CryptoMetrics
treasury_corr = metrics.calculate_corr_treasury(df=df, coin='BTCUSDT')
treasury_corr

[*********************100%***********************]  1 of 1 completed


,date,BTCUSDT,treasury_rate,treasury_corr_BTCUSDT,treasury_corr_buy_BTCUSDT,treasury_corr_sell_BTCUSDT
0,2024-11-27,95863.11,4.385,NaN,False,False
1,2024-11-28,95643.98,4.385,NaN,False,False
2,2024-11-29,97460.00,4.373,NaN,False,False
3,2024-11-30,96407.99,4.373,NaN,False,False
4,2024-12-01,97185.18,4.373,NaN,False,False
...,...,...,...,...,...,...
95,2025-03-02,94270.00,4.193,0.081990,False,False
96,2025-03-03,86220.61,4.193,0.127204,False,False
97,2025-03-04,87281.98,4.195,0.142969,False,False
98,2025-03-05,90606.01,4.205,0.168629,False,False


In [7]:
px.line(treasury_corr, x='date', y='treasury_corr_BTCUSDT')

In [8]:
fig = px.line(
    treasury_corr, 
    x='date', 
    y='BTCUSDT', 
    title='BTCUSDT correlation with interest rate USA', 
    labels= {
        'BTCUSDT': 'BTCUSDT Price', 
        'date': 'Date'
    }
)

# Add green dots for rsi_buy_BTCUSDT
fig.add_scatter(x=treasury_corr[treasury_corr['treasury_corr_buy_BTCUSDT'] == 1]['date'], 
                y=treasury_corr[treasury_corr['treasury_corr_buy_BTCUSDT'] == 1]['BTCUSDT'], 
                mode='markers', 
                marker=dict(color='green', size=10), 
                name='Treasury rate correlation buy signal')

# Add red dots for rsi_sell_BTCUSDT
fig.add_scatter(x=treasury_corr[treasury_corr['treasury_corr_sell_BTCUSDT'] == 1]['date'], 
                y=treasury_corr[treasury_corr['treasury_corr_sell_BTCUSDT'] == 1]['BTCUSDT'], 
                mode='markers', 
                marker=dict(color='red', size=10), 
                name='Treasury rate correlation sell signal')

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/4/46/Bitcoin.svg",
        xref="paper", yref="paper",
        x=1, y=1.2,
        sizex=0.15, sizey=0.15,
        xanchor="right", yanchor="bottom"
    )
)
fig.show()

### Bollinger Bands:

In [9]:
bb = metrics.calculate_bollinger_bands(df=df, coin='BTCUSDT')
bb

,date,BTCUSDT,std_BTCUSDT,mean_BTCUSDT,top_band_BTCUSDT,bottom_band_BTCUSDT,bb_buy_BTCUSDT,bb_sell_BTCUSDT
0,2024-11-27,95863.11,NaN,95863.110000,NaN,NaN,0,0
1,2024-11-28,95643.98,154.948309,95753.545000,95985.967463,95521.122537,0,0
2,2024-11-29,97460.00,991.295808,96322.363333,97809.307045,94835.419622,0,0
3,2024-11-30,96407.99,810.521170,96343.770000,97559.551755,95127.988245,0,0
4,2024-12-01,97185.18,796.431154,96512.052000,97706.698731,95317.405269,0,0
...,...,...,...,...,...,...,...,...
95,2025-03-02,94270.00,5235.870087,92359.109333,100212.914463,84505.304203,0,0
96,2025-03-03,86220.61,5350.707544,91699.275333,99725.336650,83673.214017,0,0
97,2025-03-04,87281.98,5337.647679,91132.740667,99139.212186,83126.269148,0,0
98,2025-03-05,90606.01,5188.110818,90795.025333,98577.191560,83012.859106,0,0


In [10]:
fig = px.line(
    bb, 
    x='date', 
    y='BTCUSDT', 
    title='Bollinger Bands for BTCUSDT', 
    labels= {
        'BTCUSDT': 'BTCUSDT Price', 
        'date': 'Date'
    }
)

# Add green dots for rsi_buy_BTCUSDT
fig.add_scatter(x=bb[bb['bb_buy_BTCUSDT'] == 1]['date'], 
                y=bb[bb['bb_buy_BTCUSDT'] == 1]['BTCUSDT'], 
                mode='markers', 
                marker=dict(color='green', size=10), 
                name='Bollinger Bands buy signal')

# Add red dots for rsi_sell_BTCUSDT
fig.add_scatter(x=bb[bb['bb_sell_BTCUSDT'] == 1]['date'], 
                y=bb[bb['bb_sell_BTCUSDT'] == 1]['BTCUSDT'], 
                mode='markers', 
                marker=dict(color='red', size=10), 
                name='Bollinger Bands sell signal')

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/4/46/Bitcoin.svg",
        xref="paper", yref="paper",
        x=1, y=1.2,
        sizex=0.15, sizey=0.15,
        xanchor="right", yanchor="bottom"
    )
)
fig.show()

### Mean Average Convergence Divergence (MACD):

In [11]:
macd = metrics.calculate_macd(df=df, coin='BTCUSDT')
macd

,date,BTCUSDT,EMA12_BTCUSDT,EMA26_BTCUSDT,MACD_BTCUSDT,Signal_Line_BTCUSDT,MACD_buy_BTCUSDT,MACD_sell_BTCUSDT
0,2024-11-27,95863.11,95863.110000,95863.110000,0.000000,0.000000,0,0
1,2024-11-28,95643.98,95829.397692,95846.878148,-17.480456,-3.496091,0,1
2,2024-11-29,97460.00,96080.259586,95966.368656,113.890930,19.981313,1,0
3,2024-11-30,96407.99,96130.679650,95999.081348,131.598302,42.304711,0,0
4,2024-12-01,97185.18,96292.910473,96086.940507,205.969965,75.037762,0,0
...,...,...,...,...,...,...,...,...
95,2025-03-02,94270.00,90689.146166,93515.617254,-2826.471088,-2378.614706,0,0
96,2025-03-03,86220.61,90001.679063,92975.246346,-2973.567283,-2497.605222,0,0
97,2025-03-04,87281.98,89583.263823,92553.522913,-2970.259090,-2592.135995,0,0
98,2025-03-05,90606.01,89740.609388,92409.262697,-2668.653309,-2607.439458,0,0


In [12]:
fig = px.line(
    macd, 
    x='date', 
    y='BTCUSDT', 
    title='MACD for BTCUSDT', 
    labels= {
        'BTCUSDT': 'BTCUSDT Price', 
        'date': 'Date'
    }
)

# Add green dots for rsi_buy_BTCUSDT
fig.add_scatter(x=macd[macd['MACD_buy_BTCUSDT'] == 1]['date'], 
                y=macd[macd['MACD_buy_BTCUSDT'] == 1]['BTCUSDT'], 
                mode='markers', 
                marker=dict(color='green', size=10), 
                name='MACD buy signal')

# Add red dots for rsi_sell_BTCUSDT
fig.add_scatter(x=macd[macd['MACD_sell_BTCUSDT'] == 1]['date'], 
                y=macd[macd['MACD_sell_BTCUSDT'] == 1]['BTCUSDT'], 
                mode='markers', 
                marker=dict(color='red', size=10), 
                name='MACD sell signal')

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/4/46/Bitcoin.svg",
        xref="paper", yref="paper",
        x=1, y=1.2,
        sizex=0.15, sizey=0.15,
        xanchor="right", yanchor="bottom"
    )
)
fig.show()

### Final buy setup:

In [13]:
df_buy = metrics.set_buy(df, coin='BTCUSDT')
df_buy

[*********************100%***********************]  1 of 1 completed


,date,BTCUSDT,buy_BTCUSDT,sell_BTCUSDT
0,2024-11-27,95863.11,0,0
1,2024-11-28,95643.98,0,0
2,2024-11-29,97460.00,0,0
3,2024-11-30,96407.99,0,0
4,2024-12-01,97185.18,0,0
...,...,...,...,...
95,2025-03-02,94270.00,0,0
96,2025-03-03,86220.61,0,0
97,2025-03-04,87281.98,0,0
98,2025-03-05,90606.01,0,0


In [14]:
fig = px.line(
    df_buy, 
    x='date', 
    y='BTCUSDT', 
    title='Buy setup for BTCUSDT', 
    labels= {
        'BTCUSDT': 'BTCUSDT Price', 
        'date': 'Date'
    }
)

# Add green dots for rsi_buy_BTCUSDT
fig.add_scatter(x=df_buy[df_buy['buy_BTCUSDT'] == 1]['date'], 
                y=df_buy[df_buy['buy_BTCUSDT'] == 1]['BTCUSDT'], 
                mode='markers', 
                marker=dict(color='green', size=10), 
                name='Buy signal')

# Add red dots for rsi_sell_BTCUSDT
fig.add_scatter(x=df_buy[df_buy['sell_BTCUSDT'] == 1]['date'], 
                y=df_buy[df_buy['sell_BTCUSDT'] == 1]['BTCUSDT'], 
                mode='markers', 
                marker=dict(color='red', size=10), 
                name='Sell signal')

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/4/46/Bitcoin.svg",
        xref="paper", yref="paper",
        x=1, y=1.2,
        sizex=0.15, sizey=0.15,
        xanchor="right", yanchor="bottom"
    )
)
fig.show()

## Evaluate models returns:

To evaluate the model return, it is possible to use a simulation of a bot working on the operations of trading for a given historic period of the coin price. Based on that, every trade will be set to operate with 10 dolars. By the end of the period, it will be computed the return of that.

Every buy will be 10 dollars
Every sell will be 10 dollars

The start value for investment will be 1000 dollars.

In [15]:
from src.simulations import simulate_model_trader

final_trader = simulate_model_trader(df=df_buy, initial_capital=1000, trade_value=10, coin='BTCUSDT')
print(f"Final balance: {final_trader['final_balance']:.2f}")

Initial Capital: $1000.00
Coin Holdings: 0.00035530 BTC
Coin Holdings in dollars: $32.43
Total Invested: $30.00
Final Balance: $1002.43
Total Profit/Loss: $2.43
Final balance: 1002.43


### Change of strategy:

Instead of selling the BTC, try just buying when the model advises to do:

In [16]:
from src.simulations import simulate_model_buyer

final_buyer = simulate_model_buyer(df_buy, initial_capital=1000, trade_value=10, coin="BTCUSDT")
print(f"Final balance: ${final_buyer['final_balance']:.2f}")

Initial Capital: $1000.00
Coin Holdings: 0.00035530 BTC
Coin Holdings in dollars: $32.43
Total Invested: $30.00
Final Balance: $1002.43
Total Profit/Loss: $2.43
Final balance: $1002.43


### Comparison with DCA:

In [17]:
from src.simulations import simulate_dca

final_dca = simulate_dca(df_buy, initial_capital=1000, trade_value=10, coin="BTCUSDT", investment_interval='30D')
print(f"Final balance: ${final_dca['final_balance']:.2f}")

Initial Capital: $1000.00
Coin Holdings: 0.00042057 BTC
Coin Holdings in dolars: $38.39
Total Invested: $40.00
Final Balance: $998.39
Total Profit/Loss: $-1.61
Final balance: $998.39


### Calculating ROI of different strategies:

$$ ROI = (\frac{\text{Final Balance} - \text{Initial Capital}}{\text{Initial Capital}})\times 100$$

In [18]:
from src.simulations import roi

roi_trader = roi(invested_amount= final_trader['total_invested'], final_balance= final_trader['coin_holdings_dollars'])
roi_buyer = roi(invested_amount = final_buyer['total_invested'], final_balance= final_buyer['coin_holdings_dollars'])
roi_dca = roi(invested_amount = final_dca['total_invested'], final_balance= final_dca['coin_holdings_dollars'])

print(f"Trader ROI: {roi_trader:.2f}%")
print(f"Buyer ROI: {roi_buyer:.2f}%")
print(f"DCA ROI: {roi_dca:.2f}%")

Trader ROI: 8.11%
Buyer ROI: 8.11%
DCA ROI: -4.02%


### Test for variable dates:

In [28]:
COIN = 'BTCUSDT'
LOOKBACK = 365
df = get_historical_data(
    coin=COIN,
    lookback=LOOKBACK
)

df_buy = metrics.set_buy(df, coin=COIN)

buyer_output = simulate_model_buyer(
    df_buy,
    initial_capital=1000,
    trade_value=10,
    coin=COIN
)

roi_buyer_variable = roi(invested_amount=buyer_output['total_invested'], final_balance=buyer_output['coin_holdings_dollars'])
print(f"ROI {roi_buyer_variable:.2f}%")

[*********************100%***********************]  1 of 1 completed

Initial Capital: $1000.00
Coin Holdings: 0.00291829 BTC
Coin Holdings in dollars: $266.32
Total Invested: $180.00
Final Balance: $1086.32
Total Profit/Loss: $86.32
ROI 47.96%
